In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from linearmodels.panel import PanelOLS
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler



In [ ]:

# 0) Read and preprocessing
panel = pd.read_csv("output/Final_panel_data.csv", dtype={"lsoa21cd": str})
panel["date"] = pd.to_datetime(panel["date"], errors="coerce")

# Distance column
DIST_COL = "d_net_min_km"
assert DIST_COL in panel.columns, f"Missing column {DIST_COL}"

# Area & Population density
if "Area Sq Km" in panel.columns:
    panel["area_sq_km"] = panel["Area Sq Km"]
elif "area_sq_km" not in panel.columns:
    raise ValueError("The area column was not found!")
panel["pop_density"] = panel["population"] / panel["area_sq_km"]
panel.loc[~np.isfinite(panel["pop_density"]), "pop_density"] = np.nan

panel = panel[(panel["median_price"] > 0) & panel["date"].notna()].copy()

# Main specification: Empty control variable (FE only)
BASE_CTRLS = []  

# 1) Constructing DID samples
def build_group_did_sample(panel: pd.DataFrame,
                           target_year: int,
                           t1_km: float = 1.0,
                           t2_km: float = 2.0,
                           control_scope: str = "cohort",
                           window_quarters: int | None = None):
    
    cohort = panel[panel["announcement_year"] == target_year].copy()
    cohort["treat"] = (cohort[DIST_COL] <= t1_km).astype(int)

    if control_scope == "cohort":
        control = cohort[cohort[DIST_COL] >= t2_km].copy()
    elif control_scope == "all":
        control = panel[panel[DIST_COL] >= t2_km].copy()
        control["treat"] = 0
    else:
        raise ValueError("control_scope must be 'cohort' or 'all'.")

    treated = cohort[cohort["treat"] == 1].copy()
    did = pd.concat([treated, control], ignore_index=True)

    # event time
    event_date = pd.Timestamp(f"{int(target_year)}-03-01")
    did["event_time"] = event_date

    # baseline DID 
    did["post"] = (did["date"] >= did["event_time"]).astype(int)
    did["did"] = did["post"] * did["treat"]
    did["log_price"] = np.log(did["median_price"])

    # windows
    if window_quarters is not None:
        did["date_q"] = pd.PeriodIndex(did["date"], freq="Q")
        did["event_q"] = pd.PeriodIndex(did["event_time"], freq="Q")
        did["rel_q"] = did["date_q"].astype(int) - did["event_q"].astype(int)
        did = did[(did["rel_q"] >= -window_quarters) & (did["rel_q"] <= window_quarters)]

    return did

def enforce_not_yet_treated(did_df: pd.DataFrame) -> pd.DataFrame:
    
    df = did_df.copy()
    ay = (df.groupby("lsoa21cd")["announcement_year"].max()
            .astype("Int64").replace({pd.NA: np.nan}))
    df = df.merge(ay.rename("announcement_year_self").reset_index(), on="lsoa21cd", how="left")
    df["event_time_self"] = pd.to_datetime(df["announcement_year_self"].astype("Int64").astype(str) + "-03-01",
                                           errors="coerce")
    mask = ~((df["treat"]==0) & df["event_time_self"].notna() & (df["date"] >= df["event_time_self"]))
    df = df[mask].drop(columns=["announcement_year_self","event_time_self"])
    return df

# 2) PSM (ATT/IPW) : Preprocessing features, weights, balanced diagnosis
def build_pretreatment_features(did_df: pd.DataFrame) -> pd.DataFrame:
    
    pre = did_df[did_df["date"] < did_df["event_time"]].copy()
    pre["log_price"] = np.log(pre["median_price"])

    def _trend(group):
        t = (group["date"].rank(method="first")).values
        y = group["log_price"].values
        if len(y) < 3:
            return pd.Series({"trend_price": np.nan})
        b = np.polyfit(t, y, 1)[0]
        return pd.Series({"trend_price": b})

    pre_agg = pre.groupby("lsoa21cd").agg(
        baseline_log_price_mean=("log_price", "mean"),
        total_sales_mean=("total_sales","mean") if "total_sales" in pre.columns else ("median_price","size"),
        pop_density_mean=("pop_density","mean") if "pop_density" in pre.columns else ("median_price","size"),
        share_detached_mean=("share_detached","mean") if "share_detached" in pre.columns else ("median_price","size"),
        share_semi_detached_mean=("share_semi_detached","mean") if "share_semi_detached" in pre.columns else ("median_price","size"),
        share_terraced_mean=("share_terraced","mean") if "share_terraced" in pre.columns else ("median_price","size"),
        share_flat_mean=("share_flat","mean") if "share_flat" in pre.columns else ("median_price","size"),
    ).reset_index()

    
    trend = (pre.loc[:, ["lsoa21cd","date","log_price"]]
                .groupby("lsoa21cd").apply(_trend).reset_index())
    X_pre = pre_agg.merge(trend, on="lsoa21cd", how="left")

    # The LSOA layer handles the labeling
    lsoa_role = did_df.groupby("lsoa21cd").agg(
        treat=("treat","max"),
        ann_year=("announcement_year","max")
    ).reset_index()
    X = X_pre.merge(lsoa_role, on="lsoa21cd", how="inner")
    return X

def compute_psm_att_weights(X_lsoa: pd.DataFrame,
                            feature_cols: list,
                            trim_q=(0.01,0.99),
                            stabilize=True,
                            verbose=True,
                            random_state=42) -> pd.DataFrame:
    
    cand = X_lsoa.dropna(subset=feature_cols + ["treat"]).copy()
    if cand["treat"].nunique() < 2:
        raise ValueError("The PSM candidate set does not contain both treatment and control groups.")

    # imputation
    for c in feature_cols:
        if cand[c].isna().any():
            cand[c] = cand[c].fillna(cand[c].median())

    # standard + LR
    pipe = make_pipeline(
        StandardScaler(with_mean=True, with_std=True),
        LogisticRegression(max_iter=5000, solver="lbfgs", random_state=random_state)
    )
    pipe.fit(cand[feature_cols].values, cand["treat"].values)
    cand["ps"] = pipe.predict_proba(cand[feature_cols].values)[:,1]

    # Pruning extreme ps
    lo, hi = np.quantile(cand["ps"], [trim_q[0], trim_q[1]])
    cand = cand[(cand["ps"]>=lo) & (cand["ps"]<=hi)].copy()

    # weighted（ATT）
    pt = cand["treat"].mean()
    base_w = cand["ps"] / (1.0 - cand["ps"])
    if stabilize:
        base_w = base_w * (pt / (1.0 - pt))
    cand["w_att"] = np.where(cand["treat"]==1, 1.0, base_w)

    if verbose:
        n_t = int((cand["treat"]==1).sum())
        n_c = int((cand["treat"]==0).sum())
        ess_ctrl = (cand.loc[cand["treat"]==0, "w_att"].sum()**2) / \
                   (cand.loc[cand["treat"]==0, "w_att"]**2).sum()
        print(f"[PSM] Treated LSOA: {n_t}, Candidate controls: {n_c}, Ctrl ESS: {ess_ctrl:.1f}")

    return cand[["lsoa21cd","ps","w_att","treat"]].copy()

def smd_unweighted(a, b):
    m1, m0 = np.nanmean(a), np.nanmean(b)
    s1, s0 = np.nanstd(a, ddof=1), np.nanstd(b, ddof=1)
    return (m1-m0)/np.sqrt((s1**2+s0**2)/2)

def balance_report(X_lsoa: pd.DataFrame, weights_df: pd.DataFrame, feature_cols: list):
    df = X_lsoa.merge(weights_df[["lsoa21cd","w_att","treat"]], on="lsoa21cd", how="inner", suffixes=("", "_psm"))
    treat_col = "treat_psm" if "treat_psm" in df.columns else ("treat" if "treat" in df.columns else None)
    if treat_col is None:
        raise KeyError("The treat column could not be found for balance diagnosis.")

    print("\n[Balance] Standardized mean difference (unweighted) :")
    for c in feature_cols:
        t = df.loc[df[treat_col]==1, c]; c0 = df.loc[df[treat_col]==0, c]
        print(f"  SMD({c}): {smd_unweighted(t, c0):.3f}")

    print("\n[Balance] Comparison of weighted means (Control weighted by w_att) :")
    for c in feature_cols:
        mt = df.loc[df[treat_col]==1, c].mean()
        wc = df.loc[df[treat_col]==0, "w_att"]; xc = df.loc[df[treat_col]==0, c]
        mc = np.average(xc, weights=wc) if wc.notna().any() else np.nan
        print(f"  Mean {c}: Treated={mt:.3f} | Control(w)={mc:.3f}")

# 3) DID/dose DID/Placebo without control variables
def run_baseline_did(did_df: pd.DataFrame, weights_col: str | None = None):
    df = did_df.copy().set_index(["lsoa21cd","date"]).sort_index()
    exog = df[["did"]]  # Only DID，no controls
    weights = None
    if weights_col and (weights_col in df.columns):
        w = df[weights_col].astype(float).replace([np.inf,-np.inf], np.nan).fillna(0.0)
        w[w<0] = 0.0
        weights = w
    mod = PanelOLS(df["log_price"], exog, entity_effects=True, time_effects=True,
                   drop_absorbed=True, weights=weights)
    res = mod.fit(cov_type="clustered", cluster_entity=True)
    return res

def run_dose_did(did_df: pd.DataFrame, center_at_km: float = 1.0, weights_col: str | None = None):
    df = did_df.copy()
    df["d_centered"] = df[DIST_COL] - center_at_km
    df["did_cont"] = df["did"] * df["d_centered"]
    df = df.set_index(["lsoa21cd","date"]).sort_index()
    exog = df[["did","did_cont"]]  
    weights = None
    if weights_col and (weights_col in df.columns):
        w = df[weights_col].astype(float).replace([np.inf,-np.inf], np.nan).fillna(0.0)
        w[w<0] = 0.0
        weights = w
    mod = PanelOLS(df["log_price"], exog, entity_effects=True, time_effects=True,
                   drop_absorbed=True, weights=weights)
    res = mod.fit(cov_type="clustered", cluster_entity=True)
    return res

def run_placebo(did_df: pd.DataFrame, shift_quarters=8, weights_col: str | None = None):
    df = did_df.copy()
    event_q = pd.PeriodIndex(df["event_time"], freq="Q")
    placebo_q = event_q - shift_quarters
    df["placebo_event_time"] = placebo_q.to_timestamp()
    df["placebo_post"] = (df["date"] >= df["placebo_event_time"]).astype(int)
    df["placebo_did"]  = df["placebo_post"] * df["treat"]
    df["log_price"] = np.log(df["median_price"])
    df = df.set_index(["lsoa21cd","date"]).sort_index()
    exog = df[["placebo_did"]]  
    weights = None
    if weights_col and (weights_col in df.columns):
        w = df[weights_col].astype(float).replace([np.inf,-np.inf], np.nan).fillna(0.0)
        w[w<0] = 0.0
        weights = w
    mod = PanelOLS(df["log_price"], exog, entity_effects=True, time_effects=True,
                   drop_absorbed=True, weights=weights)
    res = mod.fit(cov_type="clustered", cluster_entity=True)
    return res

# 4) Treated-only Event Study
def run_event_study_treated_only(df,
                                 k_min=-8, k_max=8,
                                 plot=True, title_suffix="",
                                 weights_col=None,
                                 enforce_nyt=True,
                                 window_quarters=None):
    
    import numpy as np
    import matplotlib.pyplot as plt
    from linearmodels.panel import PanelOLS

    df = df.copy()

    # 1) event time
    if "announcement_year" not in df.columns:
        raise ValueError("The absence of announcement_year prevents the respective event time from being constructed.")
    df["announcement_year"] = pd.to_numeric(df["announcement_year"], errors="coerce").astype("Int64")
    df["event_time_self"] = pd.to_datetime(df["announcement_year"].astype(str) + "-03-01",
                                           errors="coerce")

    # 2) not-yet-treated 
    if enforce_nyt:
        df = df[~((df["treat"]==0) & df["event_time_self"].notna() & (df["date"] >= df["event_time_self"]))].copy()

    # 3) Calculate relative_q using the "quarter number" method
    date_yq  = df["date"].dt.year * 4 + df["date"].dt.quarter
    evt_mask = df["event_time_self"].notna()
    evt_yq   = pd.Series(np.nan, index=df.index, dtype="float64")
    evt_yq.loc[evt_mask] = (df.loc[evt_mask, "event_time_self"].dt.year * 4
                            + df.loc[evt_mask, "event_time_self"].dt.quarter)
    rel = date_yq - evt_yq
    df["relative_q"] = pd.to_numeric(rel, errors="coerce").astype("Int64")

    # 4) Narrow the time window
    if (window_quarters is not None) and np.isfinite(window_quarters):
        W = int(window_quarters)
        treated_in_window = (df["treat"]==1) & df["relative_q"].notna() & (df["relative_q"].between(-W, W))
        valid_dates = df.loc[treated_in_window, "date"].unique()
        df = df[df["date"].isin(valid_dates)].copy()

    # 5) Generate event virtual only for treated The control is 0
    for k in range(k_min, k_max+1):
        mask = (df["treat"]==1) & (df["relative_q"] == k)
        df[f"ev_{k}"] = mask.fillna(False).astype(int)
    ev_cols = [f"ev_{k}" for k in range(k_min, k_max+1) if k != -1]  # k=-1 baseline period

    # 6) Clean & Index & Weight
    df = df[df["median_price"] > 0].copy()
    df["log_price"] = np.log(df["median_price"])
    df = df.set_index(["lsoa21cd","date"]).sort_index()

    exog = df[ev_cols]  
    weights = None
    if weights_col and (weights_col in df.columns):
        w = df[weights_col].astype(float).replace([np.inf,-np.inf], np.nan).fillna(0.0)
        w[w<0] = 0.0
        weights = w

    # 7) Regression (TWFE)
    mod = PanelOLS(df["log_price"], exog, entity_effects=True, time_effects=True,
                   drop_absorbed=True, weights=weights)
    res = mod.fit(cov_type="clustered", cluster_entity=True)

    actual_ev = [v for v in ev_cols if v in res.params.index]
    if not actual_ev:
        print("Event items are still absorbed or under-sampled: Please check the treaty-only logic with sample coverage.")
        return None, None, None

    # 8) Pre-trend Wald（k ≤ -2）
    pre_vars = [v for v in actual_ev if int(v.split("_")[1]) <= -2]
    wald = None
    if pre_vars:
        try:
            wald = res.wald_test([f"{v} = 0" for v in pre_vars])
            print(f"[ES treated-only] Pre-trend Wald: chi2={wald.stat:.2f}, p={wald.pval:.3g}")
        except Exception as e:
            print(f" Wald failed：{e}")

    # 9) Plot
    fig = None
    if plot:
        betas = res.params.loc[actual_ev]
        ci_df = res.conf_int().loc[actual_ev]
        lower, upper = ci_df.iloc[:,0].values, ci_df.iloc[:,1].values
        ks = np.array([int(v.split("_")[1]) for v in actual_ev])
        order = np.argsort(ks)
        x, y = ks[order], betas.values[order]
        l, u = lower[order], upper[order]
        fig = plt.figure(figsize=(10,6))
        plt.plot(x, y, marker="o", label="Event-time (treated only)")
        plt.fill_between(x, l, u, alpha=0.25, label="95% CI")
        plt.axhline(0, color="black", lw=1)
        plt.axvline(0, color="red", ls="--", lw=1, label="Event (k=0)")
        ttl = "Event Study (treated-only)"
        if title_suffix: ttl += f" — " + title_suffix
        plt.title(ttl); plt.xlabel("Quarters relative to event (k)"); plt.ylabel("Effect on log(price)")
        plt.legend(); plt.tight_layout(); plt.show()

    return res, wald, fig


# 5)cohort vs all
# PSM features
psm_features = [
    "baseline_log_price_mean","trend_price",
    "total_sales_mean","pop_density_mean",
    "share_detached_mean","share_semi_detached_mean",
    "share_terraced_mean","share_flat_mean"
]

def run_full_pipeline_for_scope(control_scope: str,
                                target_year: int,
                                T1: float, T2: float,
                                window_quarters: int | None):
    print("\n" + "="*90)
    print(f"=== Running scope = {control_scope} | window = {('±'+str(window_quarters)+'Q') if window_quarters is not None else 'ALL'} ===")

    # A) create samples
    did_data = build_group_did_sample(panel, target_year, T1, T2,
                                      control_scope=control_scope,
                                      window_quarters=window_quarters)
    if control_scope == "all":
        did_data = enforce_not_yet_treated(did_data)

    # B) PSM
    X_lsoa = build_pretreatment_features(did_data)
    psm_df = compute_psm_att_weights(X_lsoa, psm_features, trim_q=(0.01,0.99),
                                     stabilize=True, verbose=True)
    balance_report(X_lsoa, psm_df, psm_features)

    # C) Merging weights
    did_w = did_data.merge(psm_df[["lsoa21cd","w_att"]], on="lsoa21cd", how="inner")
    did_w["w_att"] = did_w["w_att"].fillna(0.0)

    # D) Baseline DID
    res_base_unw = run_baseline_did(did_data, weights_col=None)
    print("\n===== Baseline DID (Unweighted, no controls) — scope:", control_scope, "=====")
    print(res_base_unw.summary)
    if "did" in res_base_unw.params.index:
        print(f"[Unweighted] Post announcement average effect: {np.exp(res_base_unw.params['did'])-1:.2%}")

    res_base_w = run_baseline_did(did_w, weights_col="w_att")
    print("\n===== Baseline DID (PSM-ATT Weighted, no controls) — scope:", control_scope, "=====")
    print(res_base_w.summary)
    if "did" in res_base_w.params.index:
        print(f"[PSM-Weighted] Post announcement average effect: {np.exp(res_base_w.params['did'])-1:.2%}")

    # E) Dose_response DID
    res_dose_unw = run_dose_did(did_data, center_at_km=1.0, weights_col=None)
    print("\n===== Dose-Response DID (Unweighted, no controls) — scope:", control_scope, "=====")
    print(res_dose_unw.summary)

    res_dose_w = run_dose_did(did_w, center_at_km=1.0, weights_col="w_att")
    print("\n===== Dose-Response DID (PSM-ATT Weighted, no controls) — scope:", control_scope, "=====")
    print(res_dose_w.summary)
    if {"did","did_cont"}.issubset(res_dose_w.params.index):
        delta = res_dose_w.params["did"]; theta = res_dose_w.params["did_cont"]
        print(f"[Dose DID | Weighted] The effect is at 1km: {np.exp(delta)-1:.2%}；Every +1km marginal change: {np.exp(theta)-1:.2%}")

    # F) Treated-only Event Study
    did_es = did_w.dropna(subset=["announcement_year"]).copy()
    res_es_w, wald_pre_w, fig_w = run_event_study_treated_only(
        did_es, k_min=-8, k_max=8,
        title_suffix=f"{target_year} cohort — {control_scope.upper()} control, PSM-weighted, no controls",
        weights_col="w_att", enforce_nyt=True,
        window_quarters=window_quarters
    )
    if res_es_w is None:
        print("[Event Study] Skip: Event virtual is absorbed or under-sampled.")

    # G) Placebo
    res_placebo_w = run_placebo(did_w, shift_quarters=8, weights_col="w_att")
    print("\n===== Placebo DID (PSM-ATT Weighted, no controls) — scope:", control_scope, "=====")
    print(res_placebo_w.summary)
    if "placebo_did" in res_placebo_w.params.index:
        print(f"[Placebo | Weighted] Effect of Placebo: {np.exp(res_placebo_w.params['placebo_did'])-1:.2%}")

    return {
        "did_data": did_data, "did_w": did_w,
        "res_base_unw": res_base_unw, "res_base_w": res_base_w,
        "res_dose_unw": res_dose_unw, "res_dose_w": res_dose_w,
        "res_es_w": res_es_w, "res_placebo_w": res_placebo_w
    }

# 6) Run !
target_year = 2006
T1, T2 = 1.0, 2.0
WINDOW_Q = 12   

results = {}
for scope in ["cohort", "all"]:
    results[scope] = run_full_pipeline_for_scope(scope, target_year, T1, T2, WINDOW_Q)


In [ ]:
# 0) Read and preprocessing
panel = pd.read_csv("output/Final_panel_data.csv", dtype={"lsoa21cd": str})
panel["date"] = pd.to_datetime(panel["date"], errors="coerce")

# Distance column
DIST_COL = "d_net_min_km"
assert DIST_COL in panel.columns, f"Missing column {DIST_COL}"

# Area & Population density
if "Area Sq Km" in panel.columns:
    panel["area_sq_km"] = panel["Area Sq Km"]
elif "area_sq_km" not in panel.columns:
    raise ValueError("The area column was not found!")
panel["pop_density"] = panel["population"] / panel["area_sq_km"]
panel.loc[~np.isfinite(panel["pop_density"]), "pop_density"] = np.nan

panel = panel[(panel["median_price"] > 0) & panel["date"].notna()].copy()

# Main specification: Empty control variable (FE only)
BASE_CTRLS = []  

# 1) Constructing DID samples
def build_group_did_sample(panel: pd.DataFrame,
                           target_year: int,
                           t1_km: float = 1.0,
                           t2_km: float = 2.0,
                           control_scope: str = "cohort",
                           window_quarters: int | None = None):
    
    cohort = panel[panel["announcement_year"] == target_year].copy()
    cohort["treat"] = (cohort[DIST_COL] <= t1_km).astype(int)

    if control_scope == "cohort":
        control = cohort[cohort[DIST_COL] >= t2_km].copy()
    elif control_scope == "all":
        control = panel[panel[DIST_COL] >= t2_km].copy()
        control["treat"] = 0
    else:
        raise ValueError("control_scope must be 'cohort' or 'all'.")

    treated = cohort[cohort["treat"] == 1].copy()
    did = pd.concat([treated, control], ignore_index=True)

    # event time
    event_date = pd.Timestamp(f"{int(target_year)}-03-01")
    did["event_time"] = event_date

    # baseline DID 
    did["post"] = (did["date"] >= did["event_time"]).astype(int)
    did["did"] = did["post"] * did["treat"]
    did["log_price"] = np.log(did["median_price"])

    # windows
    if window_quarters is not None:
        did["date_q"] = pd.PeriodIndex(did["date"], freq="Q")
        did["event_q"] = pd.PeriodIndex(did["event_time"], freq="Q")
        did["rel_q"] = did["date_q"].astype(int) - did["event_q"].astype(int)
        did = did[(did["rel_q"] >= -window_quarters) & (did["rel_q"] <= window_quarters)]

    return did

def enforce_not_yet_treated(did_df: pd.DataFrame) -> pd.DataFrame:
    
    df = did_df.copy()
    ay = (df.groupby("lsoa21cd")["announcement_year"].max()
            .astype("Int64").replace({pd.NA: np.nan}))
    df = df.merge(ay.rename("announcement_year_self").reset_index(), on="lsoa21cd", how="left")
    df["event_time_self"] = pd.to_datetime(df["announcement_year_self"].astype("Int64").astype(str) + "-03-01",
                                           errors="coerce")
    mask = ~((df["treat"]==0) & df["event_time_self"].notna() & (df["date"] >= df["event_time_self"]))
    df = df[mask].drop(columns=["announcement_year_self","event_time_self"])
    return df

# 2) PSM (ATT/IPW) : Preprocessing features, weights, balanced diagnosis
def build_pretreatment_features(did_df: pd.DataFrame) -> pd.DataFrame:
    
    pre = did_df[did_df["date"] < did_df["event_time"]].copy()
    pre["log_price"] = np.log(pre["median_price"])

    def _trend(group):
        t = (group["date"].rank(method="first")).values
        y = group["log_price"].values
        if len(y) < 3:
            return pd.Series({"trend_price": np.nan})
        b = np.polyfit(t, y, 1)[0]
        return pd.Series({"trend_price": b})

    pre_agg = pre.groupby("lsoa21cd").agg(
        baseline_log_price_mean=("log_price", "mean"),
        total_sales_mean=("total_sales","mean") if "total_sales" in pre.columns else ("median_price","size"),
        pop_density_mean=("pop_density","mean") if "pop_density" in pre.columns else ("median_price","size"),
        share_detached_mean=("share_detached","mean") if "share_detached" in pre.columns else ("median_price","size"),
        share_semi_detached_mean=("share_semi_detached","mean") if "share_semi_detached" in pre.columns else ("median_price","size"),
        share_terraced_mean=("share_terraced","mean") if "share_terraced" in pre.columns else ("median_price","size"),
        share_flat_mean=("share_flat","mean") if "share_flat" in pre.columns else ("median_price","size"),
    ).reset_index()

    
    trend = (pre.loc[:, ["lsoa21cd","date","log_price"]]
                .groupby("lsoa21cd").apply(_trend).reset_index())
    X_pre = pre_agg.merge(trend, on="lsoa21cd", how="left")

    # The LSOA layer handles the labeling
    lsoa_role = did_df.groupby("lsoa21cd").agg(
        treat=("treat","max"),
        ann_year=("announcement_year","max")
    ).reset_index()
    X = X_pre.merge(lsoa_role, on="lsoa21cd", how="inner")
    return X

def compute_psm_att_weights(X_lsoa: pd.DataFrame,
                            feature_cols: list,
                            trim_q=(0.01,0.99),
                            stabilize=True,
                            verbose=True,
                            random_state=42) -> pd.DataFrame:
    
    cand = X_lsoa.dropna(subset=feature_cols + ["treat"]).copy()
    if cand["treat"].nunique() < 2:
        raise ValueError("The PSM candidate set does not contain both treatment and control groups.")

    # imputation
    for c in feature_cols:
        if cand[c].isna().any():
            cand[c] = cand[c].fillna(cand[c].median())

    # standard + LR
    pipe = make_pipeline(
        StandardScaler(with_mean=True, with_std=True),
        LogisticRegression(max_iter=5000, solver="lbfgs", random_state=random_state)
    )
    pipe.fit(cand[feature_cols].values, cand["treat"].values)
    cand["ps"] = pipe.predict_proba(cand[feature_cols].values)[:,1]

    # Pruning extreme ps
    lo, hi = np.quantile(cand["ps"], [trim_q[0], trim_q[1]])
    cand = cand[(cand["ps"]>=lo) & (cand["ps"]<=hi)].copy()

    # weighted（ATT）
    pt = cand["treat"].mean()
    base_w = cand["ps"] / (1.0 - cand["ps"])
    if stabilize:
        base_w = base_w * (pt / (1.0 - pt))
    cand["w_att"] = np.where(cand["treat"]==1, 1.0, base_w)

    if verbose:
        n_t = int((cand["treat"]==1).sum())
        n_c = int((cand["treat"]==0).sum())
        ess_ctrl = (cand.loc[cand["treat"]==0, "w_att"].sum()**2) / \
                   (cand.loc[cand["treat"]==0, "w_att"]**2).sum()
        print(f"[PSM] Treated LSOA: {n_t}, Candidate controls: {n_c}, Ctrl ESS: {ess_ctrl:.1f}")

    return cand[["lsoa21cd","ps","w_att","treat"]].copy()

def smd_unweighted(a, b):
    m1, m0 = np.nanmean(a), np.nanmean(b)
    s1, s0 = np.nanstd(a, ddof=1), np.nanstd(b, ddof=1)
    return (m1-m0)/np.sqrt((s1**2+s0**2)/2)

def balance_report(X_lsoa: pd.DataFrame, weights_df: pd.DataFrame, feature_cols: list):
    df = X_lsoa.merge(weights_df[["lsoa21cd","w_att","treat"]], on="lsoa21cd", how="inner", suffixes=("", "_psm"))
    treat_col = "treat_psm" if "treat_psm" in df.columns else ("treat" if "treat" in df.columns else None)
    if treat_col is None:
        raise KeyError("The treat column could not be found for balance diagnosis.")

    print("\n[Balance] Standardized mean difference (unweighted) :")
    for c in feature_cols:
        t = df.loc[df[treat_col]==1, c]; c0 = df.loc[df[treat_col]==0, c]
        print(f"  SMD({c}): {smd_unweighted(t, c0):.3f}")

    print("\n[Balance] Comparison of weighted means (Control weighted by w_att) :")
    for c in feature_cols:
        mt = df.loc[df[treat_col]==1, c].mean()
        wc = df.loc[df[treat_col]==0, "w_att"]; xc = df.loc[df[treat_col]==0, c]
        mc = np.average(xc, weights=wc) if wc.notna().any() else np.nan
        print(f"  Mean {c}: Treated={mt:.3f} | Control(w)={mc:.3f}")

# 3) DID/dose DID/Placebo without control variables
def run_baseline_did(did_df: pd.DataFrame, weights_col: str | None = None):
    df = did_df.copy().set_index(["lsoa21cd","date"]).sort_index()
    exog = df[["did"]]  # Only DID，no controls
    weights = None
    if weights_col and (weights_col in df.columns):
        w = df[weights_col].astype(float).replace([np.inf,-np.inf], np.nan).fillna(0.0)
        w[w<0] = 0.0
        weights = w
    mod = PanelOLS(df["log_price"], exog, entity_effects=True, time_effects=True,
                   drop_absorbed=True, weights=weights)
    res = mod.fit(cov_type="clustered", cluster_entity=True)
    return res

def run_dose_did(did_df: pd.DataFrame, center_at_km: float = 1.0, weights_col: str | None = None):
    df = did_df.copy()
    df["d_centered"] = df[DIST_COL] - center_at_km
    df["did_cont"] = df["did"] * df["d_centered"]
    df = df.set_index(["lsoa21cd","date"]).sort_index()
    exog = df[["did","did_cont"]]  
    weights = None
    if weights_col and (weights_col in df.columns):
        w = df[weights_col].astype(float).replace([np.inf,-np.inf], np.nan).fillna(0.0)
        w[w<0] = 0.0
        weights = w
    mod = PanelOLS(df["log_price"], exog, entity_effects=True, time_effects=True,
                   drop_absorbed=True, weights=weights)
    res = mod.fit(cov_type="clustered", cluster_entity=True)
    return res

def run_placebo(did_df: pd.DataFrame, shift_quarters=8, weights_col: str | None = None):
    df = did_df.copy()
    event_q = pd.PeriodIndex(df["event_time"], freq="Q")
    placebo_q = event_q - shift_quarters
    df["placebo_event_time"] = placebo_q.to_timestamp()
    df["placebo_post"] = (df["date"] >= df["placebo_event_time"]).astype(int)
    df["placebo_did"]  = df["placebo_post"] * df["treat"]
    df["log_price"] = np.log(df["median_price"])
    df = df.set_index(["lsoa21cd","date"]).sort_index()
    exog = df[["placebo_did"]]  
    weights = None
    if weights_col and (weights_col in df.columns):
        w = df[weights_col].astype(float).replace([np.inf,-np.inf], np.nan).fillna(0.0)
        w[w<0] = 0.0
        weights = w
    mod = PanelOLS(df["log_price"], exog, entity_effects=True, time_effects=True,
                   drop_absorbed=True, weights=weights)
    res = mod.fit(cov_type="clustered", cluster_entity=True)
    return res

# 4) Treated-only Event Study
def run_event_study_treated_only(df,
                                 k_min=-8, k_max=8,
                                 plot=True, title_suffix="",
                                 weights_col=None,
                                 enforce_nyt=True,
                                 window_quarters=None):
    
    import numpy as np
    import matplotlib.pyplot as plt
    from linearmodels.panel import PanelOLS

    df = df.copy()

    # 1) event time
    if "announcement_year" not in df.columns:
        raise ValueError("The absence of announcement_year prevents the respective event time from being constructed.")
    df["announcement_year"] = pd.to_numeric(df["announcement_year"], errors="coerce").astype("Int64")
    df["event_time_self"] = pd.to_datetime(df["announcement_year"].astype(str) + "-03-01",
                                           errors="coerce")

    # 2) not-yet-treated 
    if enforce_nyt:
        df = df[~((df["treat"]==0) & df["event_time_self"].notna() & (df["date"] >= df["event_time_self"]))].copy()

    # 3) Calculate relative_q using the "quarter number" method
    date_yq  = df["date"].dt.year * 4 + df["date"].dt.quarter
    evt_mask = df["event_time_self"].notna()
    evt_yq   = pd.Series(np.nan, index=df.index, dtype="float64")
    evt_yq.loc[evt_mask] = (df.loc[evt_mask, "event_time_self"].dt.year * 4
                            + df.loc[evt_mask, "event_time_self"].dt.quarter)
    rel = date_yq - evt_yq
    df["relative_q"] = pd.to_numeric(rel, errors="coerce").astype("Int64")

    # 4) Narrow the time window
    if (window_quarters is not None) and np.isfinite(window_quarters):
        W = int(window_quarters)
        treated_in_window = (df["treat"]==1) & df["relative_q"].notna() & (df["relative_q"].between(-W, W))
        valid_dates = df.loc[treated_in_window, "date"].unique()
        df = df[df["date"].isin(valid_dates)].copy()

    # 5) Generate event virtual only for treated The control is 0
    for k in range(k_min, k_max+1):
        mask = (df["treat"]==1) & (df["relative_q"] == k)
        df[f"ev_{k}"] = mask.fillna(False).astype(int)
    ev_cols = [f"ev_{k}" for k in range(k_min, k_max+1) if k != -1]  # k=-1 baseline period

    # 6) Clean & Index & Weight
    df = df[df["median_price"] > 0].copy()
    df["log_price"] = np.log(df["median_price"])
    df = df.set_index(["lsoa21cd","date"]).sort_index()

    exog = df[ev_cols]  
    weights = None
    if weights_col and (weights_col in df.columns):
        w = df[weights_col].astype(float).replace([np.inf,-np.inf], np.nan).fillna(0.0)
        w[w<0] = 0.0
        weights = w

    # 7) Regression (TWFE)
    mod = PanelOLS(df["log_price"], exog, entity_effects=True, time_effects=True,
                   drop_absorbed=True, weights=weights)
    res = mod.fit(cov_type="clustered", cluster_entity=True)

    actual_ev = [v for v in ev_cols if v in res.params.index]
    if not actual_ev:
        print("Event items are still absorbed or under-sampled: Please check the treaty-only logic with sample coverage.")
        return None, None, None

    # 8) Pre-trend Wald（k ≤ -2）
    pre_vars = [v for v in actual_ev if int(v.split("_")[1]) <= -2]
    wald = None
    if pre_vars:
        try:
            wald = res.wald_test([f"{v} = 0" for v in pre_vars])
            print(f"[ES treated-only] Pre-trend Wald: chi2={wald.stat:.2f}, p={wald.pval:.3g}")
        except Exception as e:
            print(f" Wald failed：{e}")

    # 9) Plot
    fig = None
    if plot:
        betas = res.params.loc[actual_ev]
        ci_df = res.conf_int().loc[actual_ev]
        lower, upper = ci_df.iloc[:,0].values, ci_df.iloc[:,1].values
        ks = np.array([int(v.split("_")[1]) for v in actual_ev])
        order = np.argsort(ks)
        x, y = ks[order], betas.values[order]
        l, u = lower[order], upper[order]
        fig = plt.figure(figsize=(10,6))
        plt.plot(x, y, marker="o", label="Event-time (treated only)")
        plt.fill_between(x, l, u, alpha=0.25, label="95% CI")
        plt.axhline(0, color="black", lw=1)
        plt.axvline(0, color="red", ls="--", lw=1, label="Event (k=0)")
        ttl = "Event Study (treated-only)"
        if title_suffix: ttl += f" — " + title_suffix
        plt.title(ttl); plt.xlabel("Quarters relative to event (k)"); plt.ylabel("Effect on log(price)")
        plt.legend(); plt.tight_layout(); plt.show()

    return res, wald, fig


# 5)cohort vs all
# PSM features
psm_features = [
    "baseline_log_price_mean","trend_price",
    "total_sales_mean","pop_density_mean",
    "share_detached_mean","share_semi_detached_mean",
    "share_terraced_mean","share_flat_mean"
]

def run_full_pipeline_for_scope(control_scope: str,
                                target_year: int,
                                T1: float, T2: float,
                                window_quarters: int | None):
    print("\n" + "="*90)
    print(f"=== Running scope = {control_scope} | window = {('±'+str(window_quarters)+'Q') if window_quarters is not None else 'ALL'} ===")

    # A) create samples
    did_data = build_group_did_sample(panel, target_year, T1, T2,
                                      control_scope=control_scope,
                                      window_quarters=window_quarters)
    if control_scope == "all":
        did_data = enforce_not_yet_treated(did_data)

    # B) PSM
    X_lsoa = build_pretreatment_features(did_data)
    psm_df = compute_psm_att_weights(X_lsoa, psm_features, trim_q=(0.01,0.99),
                                     stabilize=True, verbose=True)
    balance_report(X_lsoa, psm_df, psm_features)

    # C) Merging weights
    did_w = did_data.merge(psm_df[["lsoa21cd","w_att"]], on="lsoa21cd", how="inner")
    did_w["w_att"] = did_w["w_att"].fillna(0.0)

    # D) Baseline DID
    res_base_unw = run_baseline_did(did_data, weights_col=None)
    print("\n===== Baseline DID (Unweighted, no controls) — scope:", control_scope, "=====")
    print(res_base_unw.summary)
    if "did" in res_base_unw.params.index:
        print(f"[Unweighted] Post announcement average effect: {np.exp(res_base_unw.params['did'])-1:.2%}")

    res_base_w = run_baseline_did(did_w, weights_col="w_att")
    print("\n===== Baseline DID (PSM-ATT Weighted, no controls) — scope:", control_scope, "=====")
    print(res_base_w.summary)
    if "did" in res_base_w.params.index:
        print(f"[PSM-Weighted] Post announcement average effect: {np.exp(res_base_w.params['did'])-1:.2%}")

    # E) Dose_response DID
    res_dose_unw = run_dose_did(did_data, center_at_km=1.0, weights_col=None)
    print("\n===== Dose-Response DID (Unweighted, no controls) — scope:", control_scope, "=====")
    print(res_dose_unw.summary)

    res_dose_w = run_dose_did(did_w, center_at_km=1.0, weights_col="w_att")
    print("\n===== Dose-Response DID (PSM-ATT Weighted, no controls) — scope:", control_scope, "=====")
    print(res_dose_w.summary)
    if {"did","did_cont"}.issubset(res_dose_w.params.index):
        delta = res_dose_w.params["did"]; theta = res_dose_w.params["did_cont"]
        print(f"[Dose DID | Weighted] The effect is at 1km: {np.exp(delta)-1:.2%}；Every +1km marginal change: {np.exp(theta)-1:.2%}")

    # F) Treated-only Event Study
    did_es = did_w.dropna(subset=["announcement_year"]).copy()
    res_es_w, wald_pre_w, fig_w = run_event_study_treated_only(
        did_es, k_min=-8, k_max=8,
        title_suffix=f"{target_year} cohort — {control_scope.upper()} control, PSM-weighted, no controls",
        weights_col="w_att", enforce_nyt=True,
        window_quarters=window_quarters
    )
    if res_es_w is None:
        print("[Event Study] Skip: Event virtual is absorbed or under-sampled.")

    # G) Placebo
    res_placebo_w = run_placebo(did_w, shift_quarters=8, weights_col="w_att")
    print("\n===== Placebo DID (PSM-ATT Weighted, no controls) — scope:", control_scope, "=====")
    print(res_placebo_w.summary)
    if "placebo_did" in res_placebo_w.params.index:
        print(f"[Placebo | Weighted] Effect of Placebo: {np.exp(res_placebo_w.params['placebo_did'])-1:.2%}")

    return {
        "did_data": did_data, "did_w": did_w,
        "res_base_unw": res_base_unw, "res_base_w": res_base_w,
        "res_dose_unw": res_dose_unw, "res_dose_w": res_dose_w,
        "res_es_w": res_es_w, "res_placebo_w": res_placebo_w
    }

# 6) Run !
target_year = 2009
T1, T2 = 1.0, 2.0
WINDOW_Q = 12   

results = {}
for scope in ["cohort", "all"]:
    results[scope] = run_full_pipeline_for_scope(scope, target_year, T1, T2, WINDOW_Q)


In [ ]:

# 0) Read and preprocessing
panel = pd.read_csv("output/Final_panel_data.csv", dtype={"lsoa21cd": str})
panel["date"] = pd.to_datetime(panel["date"], errors="coerce")

# Distance column
DIST_COL = "d_net_min_km"
assert DIST_COL in panel.columns, f"Missing column {DIST_COL}"

# Area & Population density
if "Area Sq Km" in panel.columns:
    panel["area_sq_km"] = panel["Area Sq Km"]
elif "area_sq_km" not in panel.columns:
    raise ValueError("The area column was not found!")
panel["pop_density"] = panel["population"] / panel["area_sq_km"]
panel.loc[~np.isfinite(panel["pop_density"]), "pop_density"] = np.nan

panel = panel[(panel["median_price"] > 0) & panel["date"].notna()].copy()

# Main specification: Empty control variable (FE only)
BASE_CTRLS = []  

# 1) Constructing DID samples
def build_group_did_sample(panel: pd.DataFrame,
                           target_year: int,
                           t1_km: float = 1.0,
                           t2_km: float = 2.0,
                           control_scope: str = "cohort",
                           window_quarters: int | None = None):
    
    cohort = panel[panel["announcement_year"] == target_year].copy()
    cohort["treat"] = (cohort[DIST_COL] <= t1_km).astype(int)

    if control_scope == "cohort":
        control = cohort[cohort[DIST_COL] >= t2_km].copy()
    elif control_scope == "all":
        control = panel[panel[DIST_COL] >= t2_km].copy()
        control["treat"] = 0
    else:
        raise ValueError("control_scope must be 'cohort' or 'all'.")

    treated = cohort[cohort["treat"] == 1].copy()
    did = pd.concat([treated, control], ignore_index=True)

    # event time
    event_date = pd.Timestamp(f"{int(target_year)}-03-01")
    did["event_time"] = event_date

    # baseline DID 
    did["post"] = (did["date"] >= did["event_time"]).astype(int)
    did["did"] = did["post"] * did["treat"]
    did["log_price"] = np.log(did["median_price"])

    # windows
    if window_quarters is not None:
        did["date_q"] = pd.PeriodIndex(did["date"], freq="Q")
        did["event_q"] = pd.PeriodIndex(did["event_time"], freq="Q")
        did["rel_q"] = did["date_q"].astype(int) - did["event_q"].astype(int)
        did = did[(did["rel_q"] >= -window_quarters) & (did["rel_q"] <= window_quarters)]

    return did

def enforce_not_yet_treated(did_df: pd.DataFrame) -> pd.DataFrame:
    
    df = did_df.copy()
    ay = (df.groupby("lsoa21cd")["announcement_year"].max()
            .astype("Int64").replace({pd.NA: np.nan}))
    df = df.merge(ay.rename("announcement_year_self").reset_index(), on="lsoa21cd", how="left")
    df["event_time_self"] = pd.to_datetime(df["announcement_year_self"].astype("Int64").astype(str) + "-03-01",
                                           errors="coerce")
    mask = ~((df["treat"]==0) & df["event_time_self"].notna() & (df["date"] >= df["event_time_self"]))
    df = df[mask].drop(columns=["announcement_year_self","event_time_self"])
    return df

# 2) PSM (ATT/IPW) : Preprocessing features, weights, balanced diagnosis
def build_pretreatment_features(did_df: pd.DataFrame) -> pd.DataFrame:
    
    pre = did_df[did_df["date"] < did_df["event_time"]].copy()
    pre["log_price"] = np.log(pre["median_price"])

    def _trend(group):
        t = (group["date"].rank(method="first")).values
        y = group["log_price"].values
        if len(y) < 3:
            return pd.Series({"trend_price": np.nan})
        b = np.polyfit(t, y, 1)[0]
        return pd.Series({"trend_price": b})

    pre_agg = pre.groupby("lsoa21cd").agg(
        baseline_log_price_mean=("log_price", "mean"),
        total_sales_mean=("total_sales","mean") if "total_sales" in pre.columns else ("median_price","size"),
        pop_density_mean=("pop_density","mean") if "pop_density" in pre.columns else ("median_price","size"),
        share_detached_mean=("share_detached","mean") if "share_detached" in pre.columns else ("median_price","size"),
        share_semi_detached_mean=("share_semi_detached","mean") if "share_semi_detached" in pre.columns else ("median_price","size"),
        share_terraced_mean=("share_terraced","mean") if "share_terraced" in pre.columns else ("median_price","size"),
        share_flat_mean=("share_flat","mean") if "share_flat" in pre.columns else ("median_price","size"),
    ).reset_index()

    
    trend = (pre.loc[:, ["lsoa21cd","date","log_price"]]
                .groupby("lsoa21cd").apply(_trend).reset_index())
    X_pre = pre_agg.merge(trend, on="lsoa21cd", how="left")

    # The LSOA layer handles the labeling
    lsoa_role = did_df.groupby("lsoa21cd").agg(
        treat=("treat","max"),
        ann_year=("announcement_year","max")
    ).reset_index()
    X = X_pre.merge(lsoa_role, on="lsoa21cd", how="inner")
    return X

def compute_psm_att_weights(X_lsoa: pd.DataFrame,
                            feature_cols: list,
                            trim_q=(0.01,0.99),
                            stabilize=True,
                            verbose=True,
                            random_state=42) -> pd.DataFrame:
    
    cand = X_lsoa.dropna(subset=feature_cols + ["treat"]).copy()
    if cand["treat"].nunique() < 2:
        raise ValueError("The PSM candidate set does not contain both treatment and control groups.")

    # imputation
    for c in feature_cols:
        if cand[c].isna().any():
            cand[c] = cand[c].fillna(cand[c].median())

    # standard + LR
    pipe = make_pipeline(
        StandardScaler(with_mean=True, with_std=True),
        LogisticRegression(max_iter=5000, solver="lbfgs", random_state=random_state)
    )
    pipe.fit(cand[feature_cols].values, cand["treat"].values)
    cand["ps"] = pipe.predict_proba(cand[feature_cols].values)[:,1]

    # Pruning extreme ps
    lo, hi = np.quantile(cand["ps"], [trim_q[0], trim_q[1]])
    cand = cand[(cand["ps"]>=lo) & (cand["ps"]<=hi)].copy()

    # weighted（ATT）
    pt = cand["treat"].mean()
    base_w = cand["ps"] / (1.0 - cand["ps"])
    if stabilize:
        base_w = base_w * (pt / (1.0 - pt))
    cand["w_att"] = np.where(cand["treat"]==1, 1.0, base_w)

    if verbose:
        n_t = int((cand["treat"]==1).sum())
        n_c = int((cand["treat"]==0).sum())
        ess_ctrl = (cand.loc[cand["treat"]==0, "w_att"].sum()**2) / \
                   (cand.loc[cand["treat"]==0, "w_att"]**2).sum()
        print(f"[PSM] Treated LSOA: {n_t}, Candidate controls: {n_c}, Ctrl ESS: {ess_ctrl:.1f}")

    return cand[["lsoa21cd","ps","w_att","treat"]].copy()

def smd_unweighted(a, b):
    m1, m0 = np.nanmean(a), np.nanmean(b)
    s1, s0 = np.nanstd(a, ddof=1), np.nanstd(b, ddof=1)
    return (m1-m0)/np.sqrt((s1**2+s0**2)/2)

def balance_report(X_lsoa: pd.DataFrame, weights_df: pd.DataFrame, feature_cols: list):
    df = X_lsoa.merge(weights_df[["lsoa21cd","w_att","treat"]], on="lsoa21cd", how="inner", suffixes=("", "_psm"))
    treat_col = "treat_psm" if "treat_psm" in df.columns else ("treat" if "treat" in df.columns else None)
    if treat_col is None:
        raise KeyError("The treat column could not be found for balance diagnosis.")

    print("\n[Balance] Standardized mean difference (unweighted) :")
    for c in feature_cols:
        t = df.loc[df[treat_col]==1, c]; c0 = df.loc[df[treat_col]==0, c]
        print(f"  SMD({c}): {smd_unweighted(t, c0):.3f}")

    print("\n[Balance] Comparison of weighted means (Control weighted by w_att) :")
    for c in feature_cols:
        mt = df.loc[df[treat_col]==1, c].mean()
        wc = df.loc[df[treat_col]==0, "w_att"]; xc = df.loc[df[treat_col]==0, c]
        mc = np.average(xc, weights=wc) if wc.notna().any() else np.nan
        print(f"  Mean {c}: Treated={mt:.3f} | Control(w)={mc:.3f}")

# 3) DID/dose DID/Placebo without control variables
def run_baseline_did(did_df: pd.DataFrame, weights_col: str | None = None):
    df = did_df.copy().set_index(["lsoa21cd","date"]).sort_index()
    exog = df[["did"]]  # Only DID，no controls
    weights = None
    if weights_col and (weights_col in df.columns):
        w = df[weights_col].astype(float).replace([np.inf,-np.inf], np.nan).fillna(0.0)
        w[w<0] = 0.0
        weights = w
    mod = PanelOLS(df["log_price"], exog, entity_effects=True, time_effects=True,
                   drop_absorbed=True, weights=weights)
    res = mod.fit(cov_type="clustered", cluster_entity=True)
    return res

def run_dose_did(did_df: pd.DataFrame, center_at_km: float = 1.0, weights_col: str | None = None):
    df = did_df.copy()
    df["d_centered"] = df[DIST_COL] - center_at_km
    df["did_cont"] = df["did"] * df["d_centered"]
    df = df.set_index(["lsoa21cd","date"]).sort_index()
    exog = df[["did","did_cont"]]  
    weights = None
    if weights_col and (weights_col in df.columns):
        w = df[weights_col].astype(float).replace([np.inf,-np.inf], np.nan).fillna(0.0)
        w[w<0] = 0.0
        weights = w
    mod = PanelOLS(df["log_price"], exog, entity_effects=True, time_effects=True,
                   drop_absorbed=True, weights=weights)
    res = mod.fit(cov_type="clustered", cluster_entity=True)
    return res

def run_placebo(did_df: pd.DataFrame, shift_quarters=8, weights_col: str | None = None):
    df = did_df.copy()
    event_q = pd.PeriodIndex(df["event_time"], freq="Q")
    placebo_q = event_q - shift_quarters
    df["placebo_event_time"] = placebo_q.to_timestamp()
    df["placebo_post"] = (df["date"] >= df["placebo_event_time"]).astype(int)
    df["placebo_did"]  = df["placebo_post"] * df["treat"]
    df["log_price"] = np.log(df["median_price"])
    df = df.set_index(["lsoa21cd","date"]).sort_index()
    exog = df[["placebo_did"]]  
    weights = None
    if weights_col and (weights_col in df.columns):
        w = df[weights_col].astype(float).replace([np.inf,-np.inf], np.nan).fillna(0.0)
        w[w<0] = 0.0
        weights = w
    mod = PanelOLS(df["log_price"], exog, entity_effects=True, time_effects=True,
                   drop_absorbed=True, weights=weights)
    res = mod.fit(cov_type="clustered", cluster_entity=True)
    return res

# 4) Treated-only Event Study
def run_event_study_treated_only(df,
                                 k_min=-8, k_max=8,
                                 plot=True, title_suffix="",
                                 weights_col=None,
                                 enforce_nyt=True,
                                 window_quarters=None):
    
    import numpy as np
    import matplotlib.pyplot as plt
    from linearmodels.panel import PanelOLS

    df = df.copy()

    # 1) event time
    if "announcement_year" not in df.columns:
        raise ValueError("The absence of announcement_year prevents the respective event time from being constructed.")
    df["announcement_year"] = pd.to_numeric(df["announcement_year"], errors="coerce").astype("Int64")
    df["event_time_self"] = pd.to_datetime(df["announcement_year"].astype(str) + "-03-01",
                                           errors="coerce")

    # 2) not-yet-treated 
    if enforce_nyt:
        df = df[~((df["treat"]==0) & df["event_time_self"].notna() & (df["date"] >= df["event_time_self"]))].copy()

    # 3) Calculate relative_q using the "quarter number" method
    date_yq  = df["date"].dt.year * 4 + df["date"].dt.quarter
    evt_mask = df["event_time_self"].notna()
    evt_yq   = pd.Series(np.nan, index=df.index, dtype="float64")
    evt_yq.loc[evt_mask] = (df.loc[evt_mask, "event_time_self"].dt.year * 4
                            + df.loc[evt_mask, "event_time_self"].dt.quarter)
    rel = date_yq - evt_yq
    df["relative_q"] = pd.to_numeric(rel, errors="coerce").astype("Int64")

    # 4) Narrow the time window
    if (window_quarters is not None) and np.isfinite(window_quarters):
        W = int(window_quarters)
        treated_in_window = (df["treat"]==1) & df["relative_q"].notna() & (df["relative_q"].between(-W, W))
        valid_dates = df.loc[treated_in_window, "date"].unique()
        df = df[df["date"].isin(valid_dates)].copy()

    # 5) Generate event virtual only for treated The control is 0
    for k in range(k_min, k_max+1):
        mask = (df["treat"]==1) & (df["relative_q"] == k)
        df[f"ev_{k}"] = mask.fillna(False).astype(int)
    ev_cols = [f"ev_{k}" for k in range(k_min, k_max+1) if k != -1]  # k=-1 baseline period

    # 6) Clean & Index & Weight
    df = df[df["median_price"] > 0].copy()
    df["log_price"] = np.log(df["median_price"])
    df = df.set_index(["lsoa21cd","date"]).sort_index()

    exog = df[ev_cols]  
    weights = None
    if weights_col and (weights_col in df.columns):
        w = df[weights_col].astype(float).replace([np.inf,-np.inf], np.nan).fillna(0.0)
        w[w<0] = 0.0
        weights = w

    # 7) Regression (TWFE)
    mod = PanelOLS(df["log_price"], exog, entity_effects=True, time_effects=True,
                   drop_absorbed=True, weights=weights)
    res = mod.fit(cov_type="clustered", cluster_entity=True)

    actual_ev = [v for v in ev_cols if v in res.params.index]
    if not actual_ev:
        print("Event items are still absorbed or under-sampled: Please check the treaty-only logic with sample coverage.")
        return None, None, None

    # 8) Pre-trend Wald（k ≤ -2）
    pre_vars = [v for v in actual_ev if int(v.split("_")[1]) <= -2]
    wald = None
    if pre_vars:
        try:
            wald = res.wald_test([f"{v} = 0" for v in pre_vars])
            print(f"[ES treated-only] Pre-trend Wald: chi2={wald.stat:.2f}, p={wald.pval:.3g}")
        except Exception as e:
            print(f" Wald failed：{e}")

    # 9) Plot
    fig = None
    if plot:
        betas = res.params.loc[actual_ev]
        ci_df = res.conf_int().loc[actual_ev]
        lower, upper = ci_df.iloc[:,0].values, ci_df.iloc[:,1].values
        ks = np.array([int(v.split("_")[1]) for v in actual_ev])
        order = np.argsort(ks)
        x, y = ks[order], betas.values[order]
        l, u = lower[order], upper[order]
        fig = plt.figure(figsize=(10,6))
        plt.plot(x, y, marker="o", label="Event-time (treated only)")
        plt.fill_between(x, l, u, alpha=0.25, label="95% CI")
        plt.axhline(0, color="black", lw=1)
        plt.axvline(0, color="red", ls="--", lw=1, label="Event (k=0)")
        ttl = "Event Study (treated-only)"
        if title_suffix: ttl += f" — " + title_suffix
        plt.title(ttl); plt.xlabel("Quarters relative to event (k)"); plt.ylabel("Effect on log(price)")
        plt.legend(); plt.tight_layout(); plt.show()

    return res, wald, fig


# 5)cohort vs all
# PSM features
psm_features = [
    "baseline_log_price_mean","trend_price",
    "total_sales_mean","pop_density_mean",
    "share_detached_mean","share_semi_detached_mean",
    "share_terraced_mean","share_flat_mean"
]

def run_full_pipeline_for_scope(control_scope: str,
                                target_year: int,
                                T1: float, T2: float,
                                window_quarters: int | None):
    print("\n" + "="*90)
    print(f"=== Running scope = {control_scope} | window = {('±'+str(window_quarters)+'Q') if window_quarters is not None else 'ALL'} ===")

    # A) create samples
    did_data = build_group_did_sample(panel, target_year, T1, T2,
                                      control_scope=control_scope,
                                      window_quarters=window_quarters)
    if control_scope == "all":
        did_data = enforce_not_yet_treated(did_data)

    # B) PSM
    X_lsoa = build_pretreatment_features(did_data)
    psm_df = compute_psm_att_weights(X_lsoa, psm_features, trim_q=(0.01,0.99),
                                     stabilize=True, verbose=True)
    balance_report(X_lsoa, psm_df, psm_features)

    # C) Merging weights
    did_w = did_data.merge(psm_df[["lsoa21cd","w_att"]], on="lsoa21cd", how="inner")
    did_w["w_att"] = did_w["w_att"].fillna(0.0)

    # D) Baseline DID
    res_base_unw = run_baseline_did(did_data, weights_col=None)
    print("\n===== Baseline DID (Unweighted, no controls) — scope:", control_scope, "=====")
    print(res_base_unw.summary)
    if "did" in res_base_unw.params.index:
        print(f"[Unweighted] Post announcement average effect: {np.exp(res_base_unw.params['did'])-1:.2%}")

    res_base_w = run_baseline_did(did_w, weights_col="w_att")
    print("\n===== Baseline DID (PSM-ATT Weighted, no controls) — scope:", control_scope, "=====")
    print(res_base_w.summary)
    if "did" in res_base_w.params.index:
        print(f"[PSM-Weighted] Post announcement average effect: {np.exp(res_base_w.params['did'])-1:.2%}")

    # E) Dose_response DID
    res_dose_unw = run_dose_did(did_data, center_at_km=1.0, weights_col=None)
    print("\n===== Dose-Response DID (Unweighted, no controls) — scope:", control_scope, "=====")
    print(res_dose_unw.summary)

    res_dose_w = run_dose_did(did_w, center_at_km=1.0, weights_col="w_att")
    print("\n===== Dose-Response DID (PSM-ATT Weighted, no controls) — scope:", control_scope, "=====")
    print(res_dose_w.summary)
    if {"did","did_cont"}.issubset(res_dose_w.params.index):
        delta = res_dose_w.params["did"]; theta = res_dose_w.params["did_cont"]
        print(f"[Dose DID | Weighted] The effect is at 1km: {np.exp(delta)-1:.2%}；Every +1km marginal change: {np.exp(theta)-1:.2%}")

    # F) Treated-only Event Study
    did_es = did_w.dropna(subset=["announcement_year"]).copy()
    res_es_w, wald_pre_w, fig_w = run_event_study_treated_only(
        did_es, k_min=-8, k_max=8,
        title_suffix=f"{target_year} cohort — {control_scope.upper()} control, PSM-weighted, no controls",
        weights_col="w_att", enforce_nyt=True,
        window_quarters=window_quarters
    )
    if res_es_w is None:
        print("[Event Study] Skip: Event virtual is absorbed or under-sampled.")

    # G) Placebo
    res_placebo_w = run_placebo(did_w, shift_quarters=8, weights_col="w_att")
    print("\n===== Placebo DID (PSM-ATT Weighted, no controls) — scope:", control_scope, "=====")
    print(res_placebo_w.summary)
    if "placebo_did" in res_placebo_w.params.index:
        print(f"[Placebo | Weighted] Effect of Placebo: {np.exp(res_placebo_w.params['placebo_did'])-1:.2%}")

    return {
        "did_data": did_data, "did_w": did_w,
        "res_base_unw": res_base_unw, "res_base_w": res_base_w,
        "res_dose_unw": res_dose_unw, "res_dose_w": res_dose_w,
        "res_es_w": res_es_w, "res_placebo_w": res_placebo_w
    }

# 6) Run !
target_year = 2015
T1, T2 = 1.0, 2.0
WINDOW_Q = 12   

results = {}
for scope in ["cohort", "all"]:
    results[scope] = run_full_pipeline_for_scope(scope, target_year, T1, T2, WINDOW_Q)
